In [ ]:
import io
import scipy
import requests
import numpy as np 
import pandas as pd

import RADWave as rwave

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
wa = rwave.waveAnalysis(altimeterURL='../dataset/IMOS_NZ.txt', bbox=[175.0,177.0,-47.0,-45.0], 
                  stime=[1985,1,1], etime=[2019,12,31])

In [ ]:
wa.processingAltimeterData(altimeter_pick='all', saveCSV = 'altimeterData.csv')

In [ ]:
wa.visualiseData(title="Altimeter data tracks offshore New Zealand", extent=[166.,179.,-48.,-42.], 
                 addcity=None, markersize=30, zoom=8,
                 fsize=(10, 9), fsave=None)

In [ ]:
wa.timeSeries()

In [ ]:
wa.plotTimeSeries(time=[1995,2019], series='H', fsize=(12, 5), fsave=None)
wa.plotTimeSeries(time=[1995,2019], series='T', fsize=(12, 5), fsave=None)

In [ ]:
wh_all = wa.seriesSeasonMonth(series='wh', time=[1995,2020], lonlat=None, fsave=None, plot=True)

In [ ]:
T_all = wa.seriesSeasonMonth(series='period', time=[1995,2019], lonlat=None, fsave=None, plot=False)

In [ ]:
time = [1998,2018] 

In [ ]:
url = "https://www.ncdc.noaa.gov/teleconnections/enso/indicators/soi/data.csv"
soi = requests.get(url).content
soi_data = pd.read_csv(io.StringIO(soi.decode('utf-8')),skiprows=1)

soi_data['year'] = soi_data['Date'] // 100
soi_data['month'] = soi_data['Date'] % 100 



In [ ]:
soi_df = soi_data.drop(soi_data[soi_data.year < time[0]].index)
soi_df = soi_df.drop(soi_df[soi_df.year > time[1]].index)

In [ ]:
soi_mean = soi_df['Value'].mean()
soi_df['anomaly'] = soi_df['Value']-soi_mean

In [ ]:
url = "https://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/aao/monthly.aao.index.b79.current.ascii"
aoi = requests.get(url).content
aoi_data = pd.read_csv(io.StringIO(aoi.decode('utf-8')),delimiter=r"\s+",header=None)

aoi_data = aoi_data.rename(columns={0:"year", 1:"month", 2:"Value"})

In [ ]:
aoi_df = aoi_data.drop(aoi_data[aoi_data.year < time[0]].index)
aoi_df = aoi_df.drop(aoi_df[aoi_df.year > time[1]].index)

In [ ]:
aoi_mean = aoi_df['Value'].mean()
aoi_df['anomaly'] = aoi_df['Value']-aoi_mean

In [ ]:
url = "http://www.nerc-bas.ac.uk/public/icd/gjma/newsam.1957.2007.txt"
sam = requests.get(url).content
sam_data = pd.read_csv(io.StringIO(sam.decode('utf-8')),delimiter=r"\s+")

sam_data = sam_data.rename(columns={"JAN":1, "FEB":2,
                      "MAR":3, "APR":4,
                      "MAY":5, "JUN":6,
                      "JUL":7, "AUG":8,
                      "SEP": 9, "OCT":10,
                      "NOV":11, "DEC":12})

sam_data = sam_data.stack().reset_index()
sam_data = sam_data.rename(columns={"level_0":"year", "level_1":"month", 0:"Value"})

In [ ]:
sam_df = sam_data.drop(sam_data[sam_data.year < time[0]].index)
sam_df = sam_df.drop(sam_df[sam_df.year > time[1]].index)

In [ ]:
sam_mean = sam_df['Value'].mean()
sam_df['anomaly'] = sam_df['Value']-sam_mean

In [ ]:
wh_data = wa.timeseries.groupby(['year', 'month'])[['period']].apply(np.mean).reset_index()
wh_df = wh_data.drop(wh_data[wh_data.year < time[0]].index)
wh_df = wh_df.drop(wh_df[wh_df.year > time[1]].index)

In [ ]:
wh_mean = wh_df['period'].mean()
wh_df['anomaly'] = wh_df['period']-wh_mean

In [ ]:
monthly_wh_soi = scipy.stats.pearsonr(soi_df['anomaly'],wh_df['anomaly']) #indicates when it is la nina, wh increases (soi la nina is when pos)
monthly_wh_soi

In [ ]:
monthly_wh_sam = scipy.stats.pearsonr(sam_df['anomaly'],wh_df['anomaly']) #indicates when it is la nina, wh increases (soi la nina is when pos)
monthly_wh_sam

In [ ]:
monthly_wh_aoi = scipy.stats.pearsonr(aoi_df['anomaly'],wh_df['anomaly']) #indicates when it is la nina, wh increases (soi la nina is when pos)
monthly_wh_aoi